In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

**LOAD DATA**

In [3]:
mask_img_path = "../input/face-mask-12k-images-dataset/Face Mask Dataset"
os.listdir(mask_img_path)

In [4]:
train_path = mask_img_path+"/Train/"
test_path = mask_img_path+"/Test/"
validation_path = mask_img_path+"/Validation/"

In [5]:
os.listdir(train_path)

In [6]:
os.listdir(test_path)

In [7]:
os.listdir(validation_path)

In [8]:
os.listdir(train_path+'WithoutMask')[0]

In [9]:
without_mask = train_path+'WithoutMask/'+'5511.png'

In [10]:
plt.imshow(plt.imread(without_mask))

In [11]:
os.listdir(train_path+"WithMask")[7]

In [12]:
with_mask = train_path+"WithMask/"+"Augmented_79_948721.png"
plt.imshow(plt.imread(with_mask))

Number Of Images

In [13]:
print("Train With Mask: {}".format(len(os.listdir(train_path+"WithMask"))))
print("Train Without Mask: {}".format(len(os.listdir(train_path+"WithoutMask"))))
print("Test With Mask: {}".format(len(os.listdir(train_path+"WithMask"))))
print("Train Without Mask: {}".format(len(os.listdir(train_path+"WithoutMask"))))

In [14]:
plt.imread(with_mask).shape

In [15]:
image_shape = (130,130,3)

Data Pre-processing

In [16]:
#image data generator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [17]:
image_gen = ImageDataGenerator(rotation_range=5,
                              width_shift_range=0.1,
                              height_shift_range=0.1,
                              shear_range=0.02,
                              zoom_range=0.1,
                              horizontal_flip=True,
                              fill_mode='nearest')

In [18]:
no_mask_img = plt.imread(without_mask)
plt.imshow(no_mask_img)

In [19]:
plt.imshow(image_gen.random_transform(no_mask_img))

In [20]:
image_gen.flow_from_directory(train_path)

In [21]:
image_gen.flow_from_directory(test_path)

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Dropout, Flatten

**MODEL**

In [23]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3,3), input_shape=image_shape, activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=64, kernel_size=(3,3), input_shape=image_shape, activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=64, kernel_size=(3,3), input_shape=image_shape, activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam',
             metrics=['accuracy'])

In [24]:
model.summary()

In [25]:
from tensorflow.keras.callbacks import EarlyStopping

In [26]:
early_stop = EarlyStopping(monitor='val_loss', patience=2)

In [27]:
batch_size = 16

In [28]:
train_image_gen = image_gen.flow_from_directory(train_path,
                                               target_size=image_shape[:2],
                                               color_mode='rgb',
                                               batch_size=batch_size,
                                               class_mode='binary')

In [29]:
test_image_gen = image_gen.flow_from_directory(test_path,
                                               target_size=image_shape[:2],
                                               color_mode='rgb',
                                               batch_size=batch_size,
                                               class_mode='binary',
                                               shuffle=False)

In [30]:
train_image_gen.class_indices

In [31]:
results = model.fit_generator(train_image_gen, epochs=40,
                              validation_data=test_image_gen,
                              callbacks=[early_stop])

In [32]:
model.save("msak_detection_40epochs.h5")

In [33]:
import pandas as pd

In [36]:
metrics = pd.DataFrame(model.history.history)
metrics.plot()

In [37]:
metrics.columns

In [38]:
metrics[['loss', 'val_loss',]].plot()

In [40]:
metrics[['accuracy', 'val_accuracy']].plot()

**Model Evaluation**

In [51]:
pred = model.predict_generator(test_image_gen)

In [52]:
predictions =  pred > 0.5

In [53]:
predictions

In [54]:
from sklearn.metrics import classification_report, confusion_matrix

In [55]:
print(classification_report(test_image_gen.classes, predictions))

In [ ]:
confusion